# Pseudogene detection test 

`Pseudofinder` looks like a useful tool for finding pseudogenes. Not only does it look for obvious features like truncated CDS (when compared to a reference), but it also using `codeml/paml` to look for genes under selection (dn/ds) as a feature. 

Overall the software isn't too bad, but needs some quality of life adjustments. 

Software: https://github.com/filip-husnik/pseudofinder 

Install via https://github.com/filip-husnik/pseudofinder/wiki/2.-Installing-Pseudofinder#easy-installation 


I had to modify the setup.sh work for me:

* using mamba instead of conda for setting up the enviroment. 
* I had to manually run the second part of adding codeml-2.ctl to the path. see snippet from `setup.sh` below

```bash

## adding codeml-2.ctl file path:
echo '#!/bin/sh'" \

export PATH=\"$(pwd):"'$PATH'\"" \

export ctl=\"$(pwd)/codeml-2.ctl\"" >> ${CONDA_PREFIX}/etc/conda/activate.d/env_vars.sh

# re-activating environment so variable and PATH changes take effect
source activate pseudofinder
```

**Complaint 1: This really should just be in Bioconda**


## Getting started

This program needs you to give it a database of true genes (as amino acid) to compare with. I am using the wgMLST gene panel from Enterobase.

```
wget https://enterobase.warwick.ac.uk/schemes/Salmonella.wgMLST/exemplar.alleles.fasta.gz 
```

These are in nucleotide, so we must convert nuc to aa. 

In [1]:
from Bio import SeqIO, Seq
import gzip 
from Bio.Data.CodonTable import TranslationError 

gfile = gzip.open("exemplar.alleles.fasta.gz", "rt")

out_file = open('sal_alleles.faa', 'w') 


number_skip = 0 
for record in SeqIO.parse(gfile, 'fasta'):
    try:
        record.seq = record.seq.translate(cds=True)
        out_file.write(record.format("fasta"))        
    except TranslationError:
        number_skip += 1
print(f'Could not translate {number_skip} seqs ')
        


Could not translate 1596 seqs 


In [2]:
!head sal_alleles.faa
!tail sal_alleles.faa
!cat  sal_alleles.faa | grep '>' | wc -l 

>STMMW_00651_1
MHEAQIRVAIAGAGGRMGRQLIQAAMAMEGVQLGAALEREGSSLLGSDAGELAGAGKSGV
IVQSSLEAVKDDFDVFIDFTRPEGTLTHLAFCRQHGKGMVIGTTGFDDAGKQAIREASQE
IAIVFAANFSVGVNVMLKLLEKAAKVMGDYSDIEIIEAHHRHKVDAPSGTALAMGEAIAG
ALDKNLKDCAVYSREGYTGERVAGTIGFATVRAGDIVGEHTAMFADIGERVEITHKASSR
MTFANGALRSALWLKTKKNGLFDMRDVLGLDVL
>STMMW_00121_1
MAKRDYYEILGVSKTAEEREIKKAYKRLAMKYHPDRNQGDKEAEAKFKEIKEAYEVLTDA
QKRAAYDQYGHAAFEQGGMGGGFGGGFNGGADFSDIFGDVFGDIFGGGRGRQRAARGADL
RYNMDLTLEEAVRGVTKEIRIPTLEECDVCHGSGAKAGTQPQTCPTCHGSGQVQMRQGFF
QDALAILRNKLVVREHYLPCVLFGDDAPTEFTVGPVTFTQNAMFFRDKKSVFRHSVDINT
NAHIKSVTSAITQGFFRENVPTPDESRKFVGEFQKRAIKIYKDYPWVASIKVTDCDEVTS
QERAIQATELAIHIIRILLGAEPTRKIRLAWSRSNALNTAHLYSDADGVIHASVGANSLG
PVGIINWYKALMKCDLELEILGSALLPIVNPIETNHLHQRLIDAINWFGDAATDSNPSSS
IVKYVSAIERLFFGKFESGRTKVFAGRIKYILDAFGCDGDHQVYDQALKVYRARSILVHG
EIYQTEDEANESICLASSLSRMCLLCSAQLYSMMQNAFDNPDALALEEIMKRIGAEGLDW
LVDAAGFHK
>ZV79_RS12785_1
MKVETISYVKKNAATLDLSEPILVTQNGVPAYVIESYDQQQERENAIALLKLLTLSEKDK
AEGRVFSKDQLLDSLED
19466


We have a gene panel of 19466 genes to play with. >:-) 

This needs to be made into a BLAST database.

In [4]:
!makeblastdb -in sal_alleles.faa -dbtype prot
!mkdir -p blastdb
!mv sal_alleles* blastdb/




Building a new DB, current time: 07/13/2022 08:31:25
New DB name:   /home/ubuntu/code/journal/pseudo/sal_alleles.faa
New DB title:  sal_alleles.faa
Sequence type: Protein
Deleted existing Protein BLAST database named /home/ubuntu/code/journal/pseudo/sal_alleles.faa
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 19466 sequences in 0.496576 seconds.




# Finding a query genome 

We also need a genome to query. I selected THE Typhi genome (CT18; NC_003198.1) as a genbank file (gbk).  As a "reference" I used a Paratyphi (ATCC 9150; NC_006511.1). 


In [3]:
!ncbi-genome-download bacteria  -S CT18,"ATCC 9150" --formats fasta
!mkdir -p input_genomes
!zcat  refseq/bacteria/GCF_000011885.1/GCF_000011885.1_ASM1188v1_genomic.fna.gz > input_genomes/Paratyphi.ASM1188v1.fasta
!zcat  refseq/bacteria/GCF_000195995.1/GCF_000195995.1_ASM19599v1_genomic.fna.gz > input_genomes/Typhi.ASM19599v1.fasta

# Reannotate with Prokka
Annoyingly pseudofinder REALLY wants to use a prokka formatted annotation. So the original annotation for these genomes (from genbank) do not work. **Complaint 2: There should be mechanism to convert old "official" annotations to something this program can handle**


We must reannotate with prokka. *Remember to use `--compliant`, because of reasons!*

In [ ]:
!prokka input_genomes/Typhi.ASM19599v1.fasta --outdir typhi_anno --force  --compliant
!prokka input_genomes/Paratyphi.ASM1188v1.fasta --outdir paratyphi_anno --force  --compliant


# Running pseudofinder

NOW, we actually get to run the program. And enjoy all the pseudo-goodness. 

In [9]:
!mkdir -p output_fast
!pseudofinder/pseudofinder.py annotate -g typhi_anno/PROKKA_07132022.gbk  -db blastdb/sal_alleles.faa -op fast_test
!mv fast_test* output_fast/

The output log shows:
```
#Input:
Initial ORFs:	5052
Initial pseudogenes:	0
Number of contigs:	3
#Output:
Inital ORFs joined:	380
Pseudogenes (total):	631
Pseudogenes (too short):	176
Pseudogenes (too long):	154
Pseudogenes (fragmented):	275
Pseudogenes (no predicted ORF):	26
Pseudogenes (high dN/dS):	0
Pseudogenes (frameshift):	0
Pseudogenes (missing start codon):	0
Pseudogenes (missing stop codon):	0
Pseudogenes (internal stop codon):	0
Pseudogenes (multiple issues):	0
Intact genes:	4342
```

This is higher than I expected, the literature (https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2652037/) reports; 

> CT18 and Ty2 each contain ~200 pseudogenes defined as genes that are putatively inactivated by mutations including nonsense SNPs, frameshifts and truncation by deletion or rearrangement. This constitutes 4.5% of Typhi genes, much higher than the frequency in Typhimurium (0.9%) or Escherichia coli K12 (0.7%). 



# Running pseudofinder with selection included

The advanced mode using a reference genome to look at selection.



In [ ]:
!mkdir -p output_full
!pseudofinder/pseudofinder.py annotate -g typhi_anno/PROKKA_07132022.gbk  --reference paratyphi_anno/PROKKA_07132022.gbk -db blastdb/sal_alleles.faa -op full_test 
!mv full_test* output_full/

2022-07-13 09:15:55	CDS extracted from:			typhi_anno/PROKKA_07132022.gbk
			Written to file:			full_test_cds.fasta.
2022-07-13 09:15:55	Intergenic regions extracted from:	typhi_anno/PROKKA_07132022.gbk
			Written to file:			full_test_intergenic.fasta.
2022-07-13 09:15:55	blastp executed with 4 threads on full_test_proteome.faa.
